In [2]:
import hashlib

In [35]:
def fdigest(filename):
    f = open(filename,mode='rb')
    m = hashlib.sha256(f.read())
    f.close()
    return m.hexdigest()

In [36]:
h = fdigest('Loads.ipynb')
h

'd5b691758babfd8d3ddd6ad3842c08c87c825361fd9eaf8ec904ac0a7faca53d'